In [1]:
import torch

In [53]:
import numpy as np
import random
import os
import torch

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [64]:
class CoralLayer(torch.nn.Module):
    '''
    Implements CORAL layer.
    Look at arxiv paper for more details: https://arxiv.org/abs/1901.07884
    This class is an adapted version from
    https://github.com/Raschka-research-group/coral-pytorch
    
    Parameters:
    - in_features: int
        Number of features for the inputs to the forward method, which
        are expected to have shape (num_examples, in_features).

    - out_dim: int
        Number of bins, it equals to the (num_classes - 1).

    - preinit_bias: bool (default=True)
        If true, it will pre-initialize the biases to descending values in
        [0, 1] range instead of initializing it to all zeros. This pre-
        initialization scheme results in faster learning and better
        generalization performance in practice.
    '''

    def __init__(self, in_features, out_dim, preinit_bias=True):
        super().__init__()
        
        self.coral_weights = torch.nn.Linear(in_features, 1, bias=False)
        
        if preinit_bias:
            coral_bias = torch.arange(out_dim, 0, -1).float() / out_dim
        else:
            coral_bias = torch.zeros(out_dim).float()
        
        self.coral_bias = torch.nn.Parameter(coral_bias)

    def forward(self, x):
        return self.coral_weights(x) + self.coral_bias

In [65]:
size_in = 128
num_classes = 10

out_dim = 9

seed_everything(42)

coral = CoralLayer(size_in, out_dim)

In [66]:
x = torch.rand(5, 128)
x.shape

torch.Size([5, 128])

In [67]:
out = coral(x)
out.shape

torch.Size([5, 9])

In [68]:
coral.coral_bias

Parameter containing:
tensor([1.0000, 0.8889, 0.7778, 0.6667, 0.5556, 0.4444, 0.3333, 0.2222, 0.1111],
       requires_grad=True)

In [69]:
out

tensor([[ 0.5753,  0.4642,  0.3531,  0.2420,  0.1309,  0.0198, -0.0913, -0.2024,
         -0.3136],
        [ 0.4229,  0.3118,  0.2007,  0.0896, -0.0215, -0.1326, -0.2437, -0.3548,
         -0.4660],
        [ 0.6500,  0.5388,  0.4277,  0.3166,  0.2055,  0.0944, -0.0167, -0.1278,
         -0.2389],
        [ 0.8418,  0.7307,  0.6196,  0.5085,  0.3974,  0.2863,  0.1752,  0.0641,
         -0.0470],
        [ 0.6750,  0.5639,  0.4528,  0.3417,  0.2306,  0.1194,  0.0083, -0.1028,
         -0.2139]], grad_fn=<AddBackward0>)

In [63]:
out

tensor([[ 0.5753,  0.4642,  0.3531,  0.2420,  0.1309,  0.0198, -0.0913, -0.2024,
         -0.3136],
        [ 0.4229,  0.3118,  0.2007,  0.0896, -0.0215, -0.1326, -0.2437, -0.3548,
         -0.4660],
        [ 0.6500,  0.5388,  0.4277,  0.3166,  0.2055,  0.0944, -0.0167, -0.1278,
         -0.2389],
        [ 0.8418,  0.7307,  0.6196,  0.5085,  0.3974,  0.2863,  0.1752,  0.0641,
         -0.0470],
        [ 0.6750,  0.5639,  0.4528,  0.3417,  0.2306,  0.1194,  0.0083, -0.1028,
         -0.2139]], grad_fn=<AddBackward0>)

In [15]:
levels = torch.rand(5, 9)

In [18]:
import torch.nn.functional as F

In [28]:
criterion = torch.nn.BCEWithLogitsLoss()

In [13]:
def coral_loss(logits, levels, importance_weights=None, reduction='mean'):
    
    if not logits.shape == levels.shape:
        raise ValueError("Please ensure that logits (%s) has the same shape as levels (%s). "
                         % (logits.shape, levels.shape))

    term1 = F.logsigmoid(logits)*levels + (F.logsigmoid(logits) - logits)*(1-levels)

    if importance_weights is not None:
        term1 *= importance_weights

    val = (-torch.sum(term1, dim=1))

    if reduction == 'mean':
        loss = torch.mean(val)
    elif reduction == 'sum':
        loss = torch.sum(val)
    elif reduction is None:
        loss = val
    else:
        s = ('Invalid value for `reduction`. Should be "mean", '
             '"sum", or None. Got %s' % reduction)
        raise ValueError(s)

    return loss

In [39]:
def my_bce(x, y):
    
    sigmoid = torch.sigmoid(x)
   
    
    loss = y*torch.log(sigmoid) + (1 - y)*torch.log(1 - sigmoid)
    
    print(loss.shape)
    
#     loss = (-torch.sum(loss, dim=1))
    
    print(loss.shape)
    
    loss = torch.mean(loss)
    
    return loss

In [40]:
my_bce(out, levels)

torch.Size([5, 9])
torch.Size([5, 9])


tensor(-0.7110, grad_fn=<MeanBackward0>)

In [41]:
criterion(out, levels)

tensor(0.7110, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [21]:
coral_loss(out, levels)

tensor(6.3988, grad_fn=<MeanBackward0>)

# Detect inconsistency

In [74]:
a = np.array([0.876, 0.345, 0.234, 0.1234])
a = np.array([0.876, 0.234, 0.345, 0.1234])

(a[:-1] - a[1:]) < 0

array([ True, False,  True])

In [84]:
a = np.array([
    [0.876, 0.345, 0.234, 0.1234],
    [0.876, 0.234, 0.345, 0.1234],
    [0.176, 0.234, 0.345, 0.8234]
])
print(a.shape)


# (a[:-1] - a[1:]) < 0
(((a[:, :-1] - a[:, 1:]) < 0).sum(1) > 0) * 1
((a[:, :-1] - a[:, 1:]) < 0).sum(1)

(3, 4)


array([0, 1, 3])

# Calculate preds

In [113]:
outputs = torch.rand(3,5) - 0.5
outputs

tensor([[ 0.2317,  0.4771, -0.2029, -0.0864,  0.1893],
        [-0.0826, -0.0981, -0.4133,  0.1343, -0.3022],
        [ 0.0182,  0.4875, -0.1539, -0.1576,  0.3017]])

In [114]:
y_prob = outputs.sigmoid()
y_prob

tensor([[0.5577, 0.6171, 0.4494, 0.4784, 0.5472],
        [0.4794, 0.4755, 0.3981, 0.5335, 0.4250],
        [0.5045, 0.6195, 0.4616, 0.4607, 0.5748]])

In [93]:
y_pred = y_prob.sum(1).round()
y_pred

tensor([3., 2., 3.])

In [94]:
y_pred = y_prob.round().sum(1)
y_pred

tensor([3., 4., 2.])

In [99]:
y_pred = (outputs>0).sum(1)
y_pred

tensor([3, 4, 2])

In [115]:
y_pred = (y_prob>0.5)
y_pred

tensor([[ True,  True, False, False,  True],
        [False, False, False,  True, False],
        [ True,  True, False, False,  True]])

In [116]:
(y_prob > 0.5).cumprod(axis=1).sum(1)

tensor([2, 0, 2])

In [117]:
y_pred = (y_prob>0.5).sum(1)
y_pred

tensor([3, 1, 3])

In [100]:
y_prob

tensor([[0.4691, 0.4518, 0.6105, 0.5658, 0.6110],
        [0.5004, 0.5252, 0.5432, 0.3838, 0.5112],
        [0.4916, 0.4304, 0.4042, 0.6089, 0.6003]])

In [103]:
torch.tensor([2,3])

tensor([2, 3])

In [106]:
out_features = 4
torch.arange(1, out_features+1, 1).float() / out_features

tensor([0.1, 0.25, 0.9, 1.09])

tensor([0.2500, 0.5000, 0.7500, 1.0000])